# **基于Bilibili UP主：[秋葉aaaki]发布的[保姆式LoRA模型一键包文件](https://www.bilibili.com/video/BV1fs4y1x7p2/)**
最核心的文件的整合代码均由主要作者[秋葉aaaki](https://github.com/Akegarasu/lora-scripts)完成。
开始前**建议阅读**：
1.   [保姆式LoRA模型一键包文件](https://www.bilibili.com/video/BV1AL411q7Ub/?spm_id_from=333.337.search-card.all.click&vd_source=e6294619bd5137ea0025a3d001e715a7)
2.   [参数心得](https://www.bilibili.com/video/BV1GM411E7vk/)
3.   [训练教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
4.   [阿里云部署教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
>
本人只是完成阿里云下的依赖安装，使用的下载源不保证长期有效。
更新内容：
可以sdxl训练了
已经完成了秋叶于阿里云的web部署，金字塔噪声，分层训练，梯度都可以使用了
神童优化器可以使用了

根据其它群友的意见对代码进行了升级

训练包版本为2023年9月份秋叶最新版

已突破抱脸封锁，所有文件均不使用抱脸下载，不会再出现下载失败的情况了，

## 一、基础依赖和训练包拉取(时间会更长)

In [ ]:
#查看是什么GPU
!nvidia-smi
#安装解压软件和下载存储工具
!apt-get update  > /dev/null 2>&1 
!apt-get install zip -y > /dev/null 2>&1
!apt install -y aria2 > /dev/null 2>&1 && echo 全部成功

In [ ]:
%cd  /mnt/workspace/
!git clone  https://ghproxy.com/https://github.com/taiczhi/liefeng.git
!mv /mnt/workspace/liefeng/* /mnt/workspace/
!rm -r /mnt/workspace/liefeng/
!cd /mnt/workspace/lora-scripts/huggingface/ && wget -c https://liblibai-online.vibrou.com/web/model/c52a9848ee316e16b59a0fe17e17edc4c767f97f652b439d9da758d6077160ae.ckpt  -O 1.zip
!unzip /mnt/workspace/lora-scripts/huggingface/1.zip -d /mnt/workspace/lora-scripts/huggingface/ > /dev/null 2>&1 && echo 解压成功
!sudo rm -r /mnt/workspace/lora-scripts/huggingface/1.zip   > /dev/null 2>&1  && echo 删除压缩包成功
!mkdir -p /mnt/workspace/lora-scripts/train/aki
!mkdir -p /mnt/workspace/lora-scripts/train/reg

In [ ]:
#安装依赖
!pip -q install torch==2.0.1 torchvision xformers triton 
!pip -q install xformers==0.0.21
!pip -q install torchvision==0.15.2
#进入目录
%cd /mnt/workspace/lora-scripts/sd-scripts/
#安装目录要求依赖
!pip -q  install --upgrade -r requirements.txt && echo 安装需求依赖成功
%cd /mnt/workspace/lora-scripts/
!pip  install --upgrade rich
!pip  install --upgrade fastapi==0.95.1
!pip  install --upgrade uvicorn==0.22.0
!pip  install --upgrade lion-pytorch==0.1.2
!pip  install --upgrade lycoris-lora==1.9.0.dev11
!pip  install --upgrade prodigyopt==1.0
!pip  install --upgrade dadaptation==3.1
!pip  install --upgrade wandb==0.15.8
!pip  install --upgrade safetensors==0.3.1
!pip  install --upgrade bitsandbytes==0.41.1
!pip  install --upgrade pandas
!pip  install --upgrade scipy
!pip uninstall -y tensorboard tb-nightly 
!pip install tb-nightly 

In [ ]:
# 检查显卡
import torch
print('CUDA版本:',torch.version.cuda)
print('Pytorch版本:',torch.__version__)
print('显卡是否可用:','可用' if(torch.cuda.is_available()) else '不可用')
print('显卡数量:',torch.cuda.device_count())
print('是否支持BF16数字格式:','支持' if (torch.cuda.is_bf16_supported()) else '不支持不能训练sdxl')
print('当前显卡型号:',torch.cuda.get_device_name())
print('当前显卡的CUDA算力:',torch.cuda.get_device_capability())
print('当前显卡的总显存:',torch.cuda.get_device_properties(0).total_memory/1024/1024/1024,'GB')
print('是否支持TensorCore:','支持' if (torch.cuda.get_device_properties(0).major >= 7) else '不支持')
print('当前显卡的显存使用率:',torch.cuda.memory_allocated(0)/torch.cuda.get_device_properties(0).total_memory*100,'%')

## 二、模型下载（可自行改变路径）

### 只修改下方链接即可替换底膜  

默认底膜：是青龙sdxl1.0底膜base

In [ ]:
!cd /mnt/workspace/lora-scripts/sd-models/ && wget -c  https://liblibai-online.vibrou.com/web/model/1fa5725f4fe0a5fae0a840ea23725e3381213951e012aa39c3c9b70882c57015.safetensors -O down.safetensors && echo 下载成功 #下载底部模型

#### 常用模型下载

其它底膜链接：

Chilloutmix-Ni-pruned-fp32-fix：https://liblibai-online.vibrou.com/models/e7e55f800035004cf0ee31d91f874a415a2b64a4.safetensors

模型路径:/mnt/workspace/lora-scripts/sd-models/
 
麦橘v6版本模型:  

https://liblibai-online.vibrou.com/web/model/e4a30e4607faeb06b5d590b2ed8e092690c631da0b2becb6224d4bb5327104b7.safetensors 

模型路径:/mnt/workspace/lora-scripts/sd-models/

mix全能模型

https://liblibai-online.vibrou.com/web/model/f8bb2922e1dc877dc0d33ed9b9dbbdba612b8b37711cbe17d803dbc92dd65b78.safetensors 

模型路径:/mnt/workspace/lora-scripts/sd-models/

娜乌斯嘉lora模型

https://liblibai-online.vibrou.com/web/model/c932413949a815fc64378ffa6ec34944597c5ff57cd98541347c4107106fd5d5.safetensors 

模型路径:/mnt/workspace/lora-scripts/sd-models/


国风3

https://liblibai-online.vibrou.com/web/model/a83e25fe5b70bad595fe4dd6733ee35f0e3ddf8ed4041ab360f9573556e8b3e6.safetensors
 
模型路径:/mnt/workspace/lora-scripts/sd-models/

官方sdxl模型base:  

https://www.wisemodel.cn/file-proxy/LiuDQ/stable-diffusion-xl-base-1.0-stabilityai/-/raw/main/sd_xl_base_1.0.safetensors?inline=false

模型路径:/mnt/workspace/lora-scripts/sd-models/

硬纸板画模型

https://liblibai-online.vibrou.com/web/model/250cfc80d41deba9699d9c8964a8d255ef71202dba821bb18adb4760ec0a44db.safetensors

模型路径:/mnt/workspace/lora-scripts/sd-models/

可爱手办模型

https://liblibai-online.vibrou.com/web/model/39d6af08b2b67736eedac9b52f22c624fab7cc4a99c51a05a05f605f9fc43a5b.safetensors 

模型路径:/mnt/workspace/lora-scripts/sd-models/

sd1.5基础模型：

https://www.wisemodel.cn/file-proxy/LiuDQ/stable-diffusion-v1.5-runwayml/-/raw/main/v1-5-pruned.safetensors?inline=false

模型路径:/mnt/workspace/lora-scripts/sd-models/

## 三、训练集下载（可自行改变路径）

### 只修改下方wget -c链接之后即可替换数据集

In [ ]:
!cd /mnt/workspace/lora-scripts/train/aki/  && wget -c        -O 1.zip > /dev/null 2>&1 && echo 下载成功 #下载训练集

In [ ]:
!unzip /mnt/workspace/lora-scripts/train/aki/1.zip -d /mnt/workspace/lora-scripts/train/aki/ > /dev/null 2>&1 && echo 解压成功   #解压训练集压缩包
!sudo rm -r /mnt/workspace/lora-scripts/train/aki/1.zip  > /dev/null 2>&1  && echo 删除压缩包成功 #删除训练集压缩包

## 四、启动秋葉aaaki的一键式lora训练界面

In [ ]:
%cd /mnt/workspace/lora-scripts    
#跳转到UI文件
!bash run_gui.sh #界面文件点击127.0。0.0.1即可进入ui